In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[3]:


def tiktok_hash_scraper(tag_ls,url_ls,headers_ls):
    #initial empty list to save content for tag list
    text_list=[]
    for j in range(len(tag_ls)):
        #get variable from the url and header list
        url=url_ls[j]
        headers=headers_ls[j]

        #request data from the url
        response=requests.get(url,headers=headers)
        response =json.loads(response.text)
        hash_df=response['itemList']
        #loop through item list to extract necessary feature
        for i in hash_df:
            tmp_ls=[]
#            tmp_ls.append(i['id'])
            tmp_ls.append(f"https://www.tiktok.com/@{i['author']['uniqueId']}/video/{i['id']}")
            tmp_ls.append(tag_ls[j])
            tmp_ls.append(dt.datetime.fromtimestamp(i['createTime']).strftime('%Y-%m-%d'))
            tmp_ls.append(i['author']['uniqueId'])
            tmp_ls.append(i['stats']['diggCount'])
            tmp_ls.append(i['stats']['commentCount'])
            tmp_ls.append(i['stats']['playCount'])
            tmp_ls.append(i['stats']['shareCount'])
            text_list.append(tmp_ls)
    #convert to numpy array then to dataframe and change data type as needed
    array=np.array(text_list)
    array=pd.DataFrame(array,columns=['post_id', 'hash_tag','date','user_id','heart_count','comment_count','play_count','share_count'])
    array=array.astype({'heart_count': 'int32','comment_count': 'int32','play_count': 'int32','share_count': 'int32'})
    array['date']=pd.to_datetime(array['date'])
    time_boundary=pd.to_datetime((date.today() - timedelta(days = 90)).strftime('%Y-%m-%d'))
    #subset based on time
    array=array.loc[(array['date'] >= time_boundary)]
    
    return array

    


# In[4]:


def uploader(df,datatable,if_exists):
    #credentials
    POSTGRES_ADDRESS = 'db.panoply.io'
    POSTGRES_PORT = '5439'
    POSTGRES_USERNAME = '############################'
    POSTGRES_PASSWORD = '############################'
    POSTGRES_DBNAME = '############################'
    #data base form
    postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.
                    format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD, 
                        ipaddress=POSTGRES_ADDRESS, 
                        port=POSTGRES_PORT, 
                        dbname=POSTGRES_DBNAME))
    #connect and creat engine
    cnx = create_engine(postgres_str)
    #inject dataframe to datatable
    df.to_sql(datatable, con=cnx, if_exists=if_exists,index=False)
    #query to drop duplicate by window function row_number()
    drop_duplicate_query="""CREATE TABLE "public"."marketing_influencer_tiktok_hastagpost_weekly01" as
select q.*
from
(select 
h.post_id,
h.hash_tag,
h.user_id,
h.date,
h.heart_count,
h.comment_count,
h.play_count,
h.share_count,
h.update_time,
row_number() over (partition by h.post_id order by date desc) as seqnum
      from "############################" h) q
      where q.seqnum=1
      order by q.post_id;
      
DROP TABLE "public"."marketing_influencer_tiktok_hastagpost_weekly";

ALTER TABLE ######################################################## RENAME TO ############################;

select * from "public"."############################" limit 1"""
    drop_duplicate = pd.read_sql_query(drop_duplicate_query, cnx)


# In[5]:


if __name__ == '__main__':
    import datetime
    if datetime.datetime.today().weekday()==0:#modify to 0 for deployment
        print('begin')
        import json
        import requests
        import datetime as dt
        import numpy as np
        import pandas as pd

        import sqlalchemy
        import psycopg2
        from sqlalchemy import create_engine

        from datetime import date, datetime, timedelta
        #tag to be scraped
        tag_ls=['############################']
        #url extracted from part tiktok_api generated by playwright do not touch 
        url_ls=['#################################'
                ]
        #header extracted from part tiktok_api generated by playwright do not touch 
        headers_ls=[{'############################'},
                     {'############################'},
                     {'############################'},
                     {'############################'},
                     {'############################'}
                    ]
        #call function
        df=tiktok_hash_scraper(tag_ls=tag_ls,url_ls=url_ls,headers_ls=headers_ls)
        datatable='############################'
        if_exists='append'
        #set update time
        df['update_time']=pd.to_datetime((date.today()).strftime('%Y-%m-%d'))
        uploader(df=df,datatable=datatable,if_exists=if_exists)
        print('finished')
